In [ ]:
import rosbag
import cv2
import numpy as np
import os
import csv
import sys

In [ ]:
####################################################
########## Modify the Configuration  Here ##########
####################################################

# Replace with your .bag file path
bag_file = "./cameras.bag"  

# Replace with your Image Topics
topics = [
    "/zedx_left/camera_left/image_raw_color/compressed",  
    "/zedx_left/camera_right/image_raw_color/compressed"
]
# Replace with your output folder path.
# The script will create subfolder for every topic
out_folder = r"./"

In [ ]:
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

def extract_image_data(bag_file, topics,out_folder):
    if not os.path.isfile(bag_file):
        print(f"Error: Bag file '{bag_file}' does not exist.")
        return

    
    print("opening :",bag_file)
    bag = rosbag.Bag(bag_file, 'r')

    # Check if the topics exist in the bag file
    available_topics = bag.get_type_and_topic_info()[1].keys()
    missing_topics = [topic for topic in topics if topic not in available_topics]
    if missing_topics:
        print(f"Error: The following topics are not available in the bag file: {', '.join(missing_topics)}")
        bag.close()
        return

    #bag timing info
    start_time = bag.get_start_time()
    end_time = bag.get_end_time()
    total_duration = end_time - start_time

    csv_writers = {}
    open_csv_files = {}

    for topic in topics:
        folder_name = topic.replace('/', '_') 
        folder_name = folder_name.lstrip('_')

        images_folder = os.path.join(out_folder,folder_name, 'images')
        create_folder(images_folder)        
        csv_file_path = os.path.join(out_folder,folder_name, "metadata.csv")
        
        csvfile = open(csv_file_path, 'w', newline='')
        open_csv_files[topic] = csvfile
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['time','header.seq', 'header.stamp.secs', 'header.stamp.nsecs', 'header.frame_id', 'filename'])
        csv_writers[topic] = csvwriter

    last_time = start_time
    for topic, msg, t in bag.read_messages(topics=topics):
        folder_name = topic.replace('/', '_') 
        folder_name = folder_name.lstrip('_')

        image_filename = f"{t.to_nsec()}.jpg"
        image_file_path = os.path.join(out_folder,folder_name, 'images', image_filename)
        # Decode the image data        
        np_arr = np.frombuffer(msg.data, np.uint8)
        image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
        # Save the image
        cv2.imwrite(image_file_path, image)
        #update csv
        csv_writers[topic].writerow([t.to_sec(), msg.header.seq, msg.header.stamp.secs, msg.header.stamp.nsecs,msg.header.frame_id, image_filename])
    
        
        # Calculate progress
        current_time = t.to_sec()
        elapsed_time = current_time - start_time
        progress = (elapsed_time / total_duration) * 100

        # Print progress if enough time has passed
        if current_time - last_time >= 1:  # Update every second
            # Print overall progress on the same line
            sys.stdout.write(f"\rProcessing bag: {progress:.2f}% complete")
            sys.stdout.flush()
            last_time = current_time

    bag.close()

    sys.stdout.write(f"\rProcessing bag: 100.0% complete")
    sys.stdout.flush()

    for csvfile in open_csv_files.values():
        csvfile.close()

    
    print("\n---------------------------------------------")
    print(f"Data from topics {', '.join(topics)} saved to '{out_folder}'")
    


In [ ]:
extract_image_data(bag_file,topics,out_folder)